In [1]:
!git clone https://github.com/udacity/CarND-Camera-Calibration

fatal: destination path 'CarND-Camera-Calibration' already exists and is not an empty directory.


In [2]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob

%matplotlib qt

In [3]:
images = glob.glob('D:\AT\Camera_calib\CarND-Camera-Calibration\calibration_wide\GOPR*.jpg')

In [4]:
nx = 8
ny = 6

In [5]:
objpoints = []
imgpoints = []

objp = np.zeros([6*8, 3], np.float32)
objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1,2)

In [6]:
not_fnd = 0

for fname in images:
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
    else:
        not_fnd = not_fnd+1

In [7]:
not_fnd

1

In [8]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def persp_trans(img):
    img_coord = (img.shape[1], img.shape[0])
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    src = np.float32([
        [corners[0][0][0], corners[0][0][1]],
        [corners[40][0][0], corners[40][0][1]],
        [corners[-1][0][0], corners[-1][0][1]],
        [corners[7][0][0], corners[7][0][1]]
    ])
    
    width = np.abs(corners[7][0][0] - corners[0][0][0]) + 10
    height = np.abs(corners[40][0][1] - corners[0][0][1]) + 10
    
    dst = np.float32([
        [200, corners[0][0][1]],
        [200, corners[0][0][1] + height],
        [200 + width, corners[0][0][1] + height],
        [200 + width, corners[0][0][1]]
    ])
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    warped = cv2.warpPerspective(img, M, img_coord, flags=cv2.INTER_LINEAR)
    
    return warped

In [9]:
def cal_undistort_persp(img, objpoints, imgpoints):
    out = cal_undistort(img, objpoints, imgpoints)
    final = persp_trans(out)
    
    return final

In [10]:
idx = 0

img_p = images[idx]
img = mpimg.imread(img_p)

fin = cal_undistort_persp(img, objpoints, imgpoints)

fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Before')
ax1.imshow(img)

ax2.set_title('Tranformed')
ax2.imshow(fin)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)